## Spark Streaming to Blob Successful

In [1]:
!pip install azure-eventhub fastavro faker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.1/73.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.1/207.1 kB 7.9 MB/s eta 0:00:00


In [2]:
import os, time, subprocess

spark_version = subprocess.run(
    "curl -s https://downloads.apache.org/spark/ | grep -o 'spark-3\\.[0-9]\\+\\.[0-9]\\+' | sort -V | tail -1",
    shell=True, capture_output=True, text=True
).stdout.strip()

os.environ['SPARK_RELEASE'] = spark_version
os.environ['HADOOP_VERSION'] = 'hadoop3'
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['SPARK_HOME'] = f"/content/{spark_version}-bin-hadoop3"

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/${SPARK_RELEASE}/${SPARK_RELEASE}-bin-hadoop3.tgz
!tar xf ${SPARK_RELEASE}-bin-hadoop3.tgz
!pip install -q findspark


In [3]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import from_avro
from pyspark.sql.functions import col, from_unixtime, to_timestamp, window, session_window
import pyspark.sql.functions as F

In [4]:
spark = SparkSession.builder \
    .appName("AVRO_Streaming_EventHub") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config("spark.sql.shuffle.partitions", 4) \
    .config("spark.jars.packages",
            "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,"
            "org.apache.spark:spark-avro_2.12:3.5.0,"
            "org.apache.hadoop:hadoop-azure:3.3.1,"
            "com.microsoft.azure:azure-storage:8.6.6") \
    .master("local[*]") \
    .getOrCreate()


In [5]:
from pyspark.sql import SparkSession

In [6]:
from pyspark.sql import SparkSession

jar_dependencies = ",".join([
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0",
    "org.apache.spark:spark-avro_2.12:3.5.0",
    "org.apache.hadoop:hadoop-azure:3.3.1",
    "com.microsoft.azure:azure-storage:8.6.6"
])

account_name = "iesstsabbadbaa"
account_key = "ZT6z+TYSxF0Xdm0vOCRbIpWoBss2BxOU0EcP2UDceddHX7Kyi8gyJvjyWG5THNp2HOprCHmblb2f+AStp8mAGw=="

spark = SparkSession.builder \
    .appName("StreamingAVROFromKafka") \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config("spark.jars.packages", jar_dependencies) \
    .config(f"fs.azure.account.key.{account_name}.blob.core.windows.net", account_key) \
    .config("spark.sql.shuffle.partitions", 4) \
    .master("local[*]") \
    .getOrCreate()


In [7]:
spark.conf.set(
    "fs.azure.account.key.iesstsabbadbaa.blob.core.windows.net",
    "ZT6z+TYSxF0Xdm0vOCRbIpWoBss2BxOU0EcP2UDceddHX7Kyi8gyJvjyWG5THNp2HOprCHmblb2f+AStp8mAGw=="
)

# Event Hub (Kafka API) Settings
eventhub_namespace = 'iesstsabbadbaa-grp-06-10'
topic1_name = 'passenger_requests_10'
topic2_name = 'ride_status_10'
consumer_connection_string = "Endpoint=sb://iesstsabbadbaa-grp-06-10.servicebus.windows.net/;SharedAccessKeyName=Consumer_10;SharedAccessKey=jPZjSqQc7HIox8LooseofiUY2mg5/pO/J+AEhDJ4b50="

kafkaConf = {
    "kafka.bootstrap.servers": f"{eventhub_namespace}.servicebus.windows.net:9093",
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol": "SASL_SSL",
    "kafka.sasl.jaas.config": f'org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="{consumer_connection_string.split("SharedAccessKey=")[-1].split(";")[0]}";',
    "startingOffsets": "latest",
    "enable.auto.commit": "true",
    "groupIdPrefix": "Stream_Analytics_",
    "auto.commit.interval.ms": "5000"
}

kafkaConf_topic1 = kafkaConf.copy()
kafkaConf_topic1["subscribe"] = topic1_name

kafkaConf_topic2 = kafkaConf.copy()
kafkaConf_topic2["subscribe"] = topic2_name


In [8]:
import json

passenger_requests_schema = """
{
  "type": "record",
  "name": "PassengerRequest",
  "fields": [
    {"name": "request_id", "type": "string"},
    {"name": "user_id", "type": "string"},
    {"name": "event_type", "type": {
      "type": "enum", "name": "EventType",
      "symbols": ["request", "cancel"]
    }},
    {"name": "timestamp", "type": "long"},
    {"name": "pickup_latitude", "type": "double"},
    {"name": "pickup_longitude", "type": "double"},
    {"name": "pickup_address", "type": "string"},
    {"name": "dropoff_latitude", "type": "double"},
    {"name": "dropoff_longitude", "type": "double"},
    {"name": "dropoff_address", "type": "string"},
    {"name": "payment_method", "type": {
      "type": "enum", "name": "PaymentMethod",
      "symbols": ["credit_card", "debit_card", "cash", "mobile_payment"]
    }},
    {"name": "vehicle_type", "type": {
      "type": "enum", "name": "VehicleType",
      "symbols": ["standard", "premium", "luxury", "pool"]
    }},
    {"name": "num_passengers", "type": "int"},
    {"name": "fare_estimate", "type": "float"},
    {"name": "ride_purpose", "type": {
      "type": "enum", "name": "RidePurpose",
      "symbols": ["commute", "airport", "business", "leisure"]
    }},
    {"name": "device_type", "type": {
      "type": "enum", "name": "DeviceType",
      "symbols": ["mobile", "web"]
    }}
  ]
}
"""

ride_status_schema = """
{
  "type": "record",
  "name": "RideStatus",
  "fields": [
    {"name": "ride_id", "type": "string"},
    {"name": "request_id", "type": "string"},
    {"name": "status", "type": {
      "type": "enum", "name": "RideStatusEnum",
      "symbols": ["accepted", "ongoing", "completed", "cancelled"]
    }},
    {"name": "timestamp", "type": "long"},
    {"name": "driver_id", "type": "string"},
    {"name": "estimated_arrival_time", "type": "long"},
    {"name": "actual_arrival_time", "type": ["null", "long"], "default": null},
    {"name": "ride_duration", "type": ["null", "int"], "default": null},
    {"name": "distance_traveled", "type": ["null", "float"], "default": null},
    {"name": "fare", "type": "float"},
    {"name": "surge_multiplier", "type": "float"},
    {"name": "traffic_condition", "type": {
      "type": "enum", "name": "TrafficCondition",
      "symbols": ["low", "medium", "high"]
    }},
    {"name": "weather_condition", "type": {
      "type": "enum", "name": "WeatherCondition",
      "symbols": ["clear", "rainy", "snowy", "foggy", "stormy"]
    }},
    {"name": "day_of_week", "type": {
      "type": "enum", "name": "DayOfWeek",
      "symbols": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    }},
    {"name": "driver_rating", "type": ["null", "float"], "default": null},
    {"name": "cancellation_reason", "type": ["null", {
      "type": "enum", "name": "CancellationReason",
      "symbols": ["user_request", "driver_unavailable", "technical_issue"]
    }], "default": null},
    {"name": "payment_status", "type": {
      "type": "enum", "name": "PaymentStatus",
      "symbols": ["pending", "completed", "failed"]
    }}
  ]
}
"""

passenger_schema_str = json.dumps(json.loads(passenger_requests_schema))
rider_schema_str = json.dumps(json.loads(ride_status_schema))



In [9]:
df_passenger = spark.readStream \
    .format("kafka") \
    .options(**kafkaConf) \
    .option("subscribe", topic1_name) \
    .load()

df_passenger = df_passenger.select(from_avro(df_passenger.value, passenger_schema_str).alias("passenger"))

df_passenger = df_passenger.selectExpr(
    "passenger.request_id",
    "passenger.user_id",
    "passenger.timestamp",
    "passenger.pickup_latitude as pickup_lat",
    "passenger.pickup_longitude as pickup_lon",
    "passenger.dropoff_latitude as dropoff_lat",
    "passenger.dropoff_longitude as dropoff_lon",
    "passenger.event_type",
    "passenger.vehicle_type",
    "passenger.fare_estimate",
    "passenger.num_passengers",
    "passenger.ride_purpose",
    "passenger.device_type"
)


In [11]:
query_test_passenger = df_passenger.writeStream \
    .format("console") \
    .outputMode("append") \
    .option("truncate", False) \
    .start()

import time
time.sleep(15)

query_test_passenger.stop()



In [10]:
df_rider = spark.readStream \
    .format("kafka") \
    .options(**kafkaConf) \
    .option("subscribe", topic2_name) \
    .load()

df_rider = df_rider.select(
    from_avro(df_rider.value, rider_schema_str).alias("rider")
)

df_rider = df_rider.selectExpr(
    "rider.ride_id", "rider.request_id", "rider.status", "rider.timestamp",
    "rider.driver_id", "rider.estimated_arrival_time", "rider.actual_arrival_time",
    "rider.ride_duration", "rider.distance_traveled", "rider.fare",
    "rider.surge_multiplier", "rider.traffic_condition", "rider.weather_condition",
    "rider.day_of_week", "rider.driver_rating", "rider.cancellation_reason", "rider.payment_status"
)



In [12]:
# Start passenger stream to Azure Blob
passenger_request_query = df_passenger.writeStream.format("parquet") \
    .option("path", "wasbs://group10@iesstsabbadbaa.blob.core.windows.net/passenger_request_data") \
    .option("checkpointLocation", "wasbs://group10@iesstsabbadbaa.blob.core.windows.net/checkpoints/passenger_request_data") \
    .outputMode("append") \
    .trigger(processingTime="5 seconds") \
    .start()

# Start rider stream to Azure Blob
ride_status_query = df_rider.writeStream.format("parquet") \
    .option("path", "wasbs://group10@iesstsabbadbaa.blob.core.windows.net/ride_status_data") \
    .option("checkpointLocation", "wasbs://group10@iesstsabbadbaa.blob.core.windows.net/checkpoints/ride_status_data") \
    .outputMode("append") \
    .trigger(processingTime="5 seconds") \
    .start()

# Let both run for a set duration
import time
time.sleep(500)

# Stop streaming queries gracefully
print("🛑 Stopping active queries...")
for query in spark.streams.active:
    query.stop()

# Shutdown Spark session
spark.stop()
print("✅ All done! Data should now be in Azure Blob Storage.")


🛑 Stopping active queries...
✅ All done! Data should now be in Azure Blob Storage.
